Construct a toy demo which translate Mandarin audio file into English one

The demo demonstrates how to :
- instantiate a pre-trained NeMo models from NVIDIA NGC
- Transcribe audio with speech recognition model
- Translate text with machine translation model
- Generate audio with TTS models

# Installation

NeMo can be installed via simple pip command

In [2]:
BRANCH = 'r1.16.0'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

# install Pynini for text normalization
# ! wget https://raw.githubusercontent.com/NVIDIA/NeMo/${BRANCH}/nemo_text_processing/install_pynini.sh


DEPRECATION: git+https://github.com/NVIDIA/NeMo.git@r1.16.0#egg=nemo_toolkit[all] contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
  Cloning https://github.com/NVIDIA/NeMo.git (to revision r1.16.0) to /tmp/pip-install-t_3o2s6b/nemo-toolkit_e09a0964648d4b698dcbcb341afc4f36
  Running command git clone --filter=blob:none --quiet https://github.com/NVIDIA/NeMo.git /tmp/pip-install-t_3o2s6b/nemo-toolkit_e09a0964648d4b698dcbcb341afc4f36
  Running command git checkout -b r1.16.0 --track origin/r1.16.0
  Switched to a new branch 'r1.16.0'
  Branch 'r1.16.0' set up to track remote branch 'r1.16.0' from 'origin'.
  Resolved https://github.com/NVIDIA/NeMo.git to commit 163111870e78a2a7224953a89b6af5ac71cb8ed9
  Preparing metadata (setup.py) ... done
--2023-03-20 15:02:37--  https://raw.githubusercontent

In [2]:
# ! wget https://raw.githubusercontent.com/NVIDIA/NeMo/blob/v1.16.0/nemo_text_processing/install_pynini.sh
! wget https://raw.githubusercontent.com/NVIDIA/NeMo/v1.16.0/nemo_text_processing/install_pynini.sh
! bash install_pynini.sh


--2023-03-21 11:37:22--  https://raw.githubusercontent.com/NVIDIA/NeMo/v1.16.0/nemo_text_processing/install_pynini.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38 [text/plain]
Saving to: ‘install_pynini.sh’

install_pynini.sh   100%[===================>]      38  --.-KB/s    in 0s      

2023-03-21 11:37:23 (1.01 MB/s) - ‘install_pynini.sh’ saved [38/38]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.3/161.3 MB 2.6 MB/s eta 0:00:0000:0100:02


In [5]:
import nemo
# Import speech recognition collection
import nemo.collections.asr as nemo_asr
import nemo.collections.nlp as nemo_nlp
import nemo.collections.tts as nemo_tts

import IPython

[NeMo W 2023-03-21 10:15:14 optimizers:54] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2023-03-21 10:15:15 experimental:27] Module <class 'nemo.collections.asr.modules.audio_modules.SpectrogramToMultichannelFeatures'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2023-03-21 10:15:16 experimental:27] Module <class 'nemo.collections.nlp.data.language_modeling.megatron.megatron_batch_samplers.MegatronPretrainingRandomBatchSampler'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2023-03-21 10:15:16 experimental:27] Module <class 'nemo.collections.nlp.models.text_normalization_as_tagging.thutmose_tagger.ThutmoseTaggerModel'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2023-03-21 10:15:16 experimental:27] Module <class 'nemo.collections.common.tokenizers.text_to_speech.tts_tokenizers.IPATokeni

# Instantiate pre-trained NeMo models

Every NeMo model has these methods:
- `list_available_models`: list all models currently available on NGC and their names
- `from_pretrained(...)`: API downloads and initialized model directly fron NGC using model name

In [6]:
# Here is an example of all CTC-based models
nemo_asr.models.EncDecCTCModel.list_available_models()
# More ASR models are available - see: nemo_asr.models.ASRModel.list_available_models()

[PretrainedModelInfo(
 	pretrained_model_name=QuartzNet15x5Base-En,
 	description=QuartzNet15x5 model trained on six datasets: LibriSpeech, Mozilla Common Voice (validated clips from en_1488h_2019-12-10), WSJ, Fisher, Switchboard, and NSC Singapore English. It was trained with Apex/Amp optimization level O1 for 600 epochs. The model achieves a WER of 3.79% on LibriSpeech dev-clean, and a WER of 10.05% on dev-other. Please visit https://ngc.nvidia.com/catalog/models/nvidia:nemospeechmodels for further details.,
 	location=https://api.ngc.nvidia.com/v2/models/nvidia/nemospeechmodels/versions/1.0.0a5/files/QuartzNet15x5Base-En.nemo
 ),
 PretrainedModelInfo(
 	pretrained_model_name=stt_en_quartznet15x5,
 	description=For details about this model, please visit https://ngc.nvidia.com/catalog/models/nvidia:nemo:stt_en_quartznet15x5,
 	location=https://api.ngc.nvidia.com/v2/models/nvidia/nemo/stt_en_quartznet15x5/versions/1.0.0rc1/files/stt_en_quartznet15x5.nemo
 ),
 PretrainedModelInfo(
 	pre

In [7]:
CUDA_LAUNCH_BLOCKING=1
# Speech Recognition model - Citrinet initially trained on Multilingual LibriSpeech English corpus, and fine-tuned on the open source Aishell-2
asr_model = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="stt_zh_citrinet_1024_gamma_0_25").cuda()


[NeMo I 2023-03-21 10:15:32 cloud:56] Found existing object /home/mat/.cache/torch/NeMo/NeMo_1.16.0/stt_zh_citrinet_1024_gamma_0_25/e4a8b1119971335507d9672e03bc80f4/stt_zh_citrinet_1024_gamma_0_25.nemo.
[NeMo I 2023-03-21 10:15:32 cloud:62] Re-using file from: /home/mat/.cache/torch/NeMo/NeMo_1.16.0/stt_zh_citrinet_1024_gamma_0_25/e4a8b1119971335507d9672e03bc80f4/stt_zh_citrinet_1024_gamma_0_25.nemo
[NeMo I 2023-03-21 10:15:32 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-03-21 10:15:32 nemo_logging:349] /home/mat/anaconda3/envs/nemo/lib/python3.9/site-packages/torch/cuda/__init__.py:88: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
      return torch._C._cuda_getDeviceCount() > 0
    
[NeMo W 2023-03-21 10:15:52 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    labels:
    - ' '
    - ''''
    - A
    - B
    - C
    - D
    - E
    - F
    - G
    - H
    - I
    - J
    - K
    - L
    - M
    - 'N'
    - O
    - P
    - Q
    - R
    - S
    - T
    - U
    - V
    - W
    - X
    - '

[NeMo I 2023-03-21 10:15:52 features:286] PADDING: 16
[NeMo I 2023-03-21 10:16:00 save_restore_connector:247] Model EncDecCTCModel was successfully restored from /home/mat/.cache/torch/NeMo/NeMo_1.16.0/stt_zh_citrinet_1024_gamma_0_25/e4a8b1119971335507d9672e03bc80f4/stt_zh_citrinet_1024_gamma_0_25.nemo.


RuntimeError: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero.

In [10]:

# Neural machine translation model
nmt_model = nemo_nlp.models.MTEncDecModel.from_pretrained(model_name="nmt_zh_en_transformer6x6").cuda()

# Spectrogram generator which text as input and produces spectrogram
spectrogram_generator = nemo_tts.models.FastPitchModel.from_pretrained(model_name="tts_en_fastpitch").cuda()

# Vocoder model which takes spectrogram and produces actual audio
vocoder = nemo_tts.models.HifiGanModel.from_pretrained(model_name="tts_hifigan").cuda()

[NeMo I 2023-03-21 09:47:53 cloud:56] Found existing object /home/mat/.cache/torch/NeMo/NeMo_1.16.0/nmt_zh_en_transformer6x6/eff3792e6f4420ba83436be889e92d79/nmt_zh_en_transformer6x6.nemo.
[NeMo I 2023-03-21 09:47:53 cloud:62] Re-using file from: /home/mat/.cache/torch/NeMo/NeMo_1.16.0/nmt_zh_en_transformer6x6/eff3792e6f4420ba83436be889e92d79/nmt_zh_en_transformer6x6.nemo
[NeMo I 2023-03-21 09:47:53 common:913] Instantiating model from pre-trained checkpoint
[NeMo I 2023-03-21 09:48:04 tokenizer_utils:179] Getting YouTokenToMeTokenizer with model: /tmp/tmpiieuvnne/tokenizer.decoder.32000.BPE.model with r2l: False.
[NeMo I 2023-03-21 09:48:04 tokenizer_utils:179] Getting YouTokenToMeTokenizer with model: /tmp/tmpiieuvnne/tokenizer.encoder.32000.BPE.model with r2l: False.


[NeMo W 2023-03-21 09:48:04 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    src_file_name: /raid/tarred_data_accaligned_16k_tokens_32k_vocab_cov_0.999/batches.tokens.16000._OP_1..144_CL_.tar
    tgt_file_name: /raid/tarred_data_accaligned_16k_tokens_32k_vocab_cov_0.999/batches.tokens.16000._OP_1..144_CL_.tar
    tokens_in_batch: 16000
    clean: true
    max_seq_length: 512
    cache_ids: false
    cache_data_per_node: false
    use_cache: false
    shuffle: true
    num_samples: -1
    drop_last: false
    pin_memory: false
    num_workers: 8
    load_from_cached_dataset: false
    reverse_lang_direction: true
    load_from_tarred_dataset: true
    metadata_path: /raid/tarred_data_accaligned_16k_tokens_32k_vocab_cov_0.999/metadata.json
    tar_shuffle_n: 100
    
[NeMo W 2023-03-21 09:48:04 modelPT:168] If you intend to do valida

RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

Get an audio sample in Mandarin

In [ ]:
# Download audio sample which we'll try
# This is a sample from MCV 6.1 Dev dataset - the model hasn't seen it before
# IMPORTANT: The audio must be mono with 16Khz sampling rate
audio_sample = 'common_voice_zh-CN_21347786.mp3'
!wget 'https://nemo-public.s3.us-east-2.amazonaws.com/zh-samples/common_voice_zh-CN_21347786.mp3'
# To listen it, click on the play button below
IPython.display.Audio(audio_sample)

Transcribe audio file

We'll use speech recognition model to convert audio into text

In [ ]:
transcribed_text = asr_model.transcribe([audio_sample])
print(transcribed_text)

Transcribe chinese text into english

NeMo's NMT models have an handy `.translate()` method

In [ ]:
english_text = nmt_model.transcribe(transcribed_text)
print(english_text)

Generate english audio from text

Speech generation from text typically has 2 steps:
- Generate spectrogram from the text, using FastPitch model
- Generate actual audio from spectrogram , using Hifigan model

In [11]:
# A helper function which combines FastPitch and HifiGan to go directly from 
# text to audio
def text_to_audio(text):
    parsed = spectrogram_generator.parse(text)
    spectrogram = spectrogram_generator.generate_spectrogram(tokens=parsed)
    audio = vocoder.convert_spectrogram_to_audio(spec=spectrogram)
    return audio.to('cpu').detach().numpy()


In [ ]:
# Listen to generated audio in English
IPython.display.Audio(text_to_audio(english_text[0]), rate=22050)

In [13]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Jun__8_16:49:14_PDT_2022
Cuda compilation tools, release 11.7, V11.7.99
Build cuda_11.7.r11.7/compiler.31442593_0
